In [1]:
import cassandra

In [2]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [3]:
# Creating the keyspace (~ database)
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS data_modelling
        WITH REPLICATION =
            {
                'class' : 'SimpleStrategy',
                'replication_factor' : 1
            }
    """)
except Exception as e:
    print(e)

In [4]:
# Connect to the keyspace
try:
    session.set_keyspace('data_modelling')
except Exception as e:
    print(e)

<h2>Questions for our data</h2>
<h4>NEED TO MOVE THROUGH THE WHERE CLAUSE IN THE LOD DIRECTION OF PRIMARY KEYS</h4>

In [5]:
query = 'CREATE TABLE IF NOT EXISTS music_library '
query += '(YEAR INT, CITY TEXT, ARTIST_NAME TEXT, ALBUM_NAME TEXT, PRIMARY KEY (YEAR, ARTIST_NAME, ALBUM_NAME))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [6]:
query = 'INSERT INTO music_library (YEAR, CITY, ARTIST_NAME, ALBUM_NAME)'
query += ' VALUES (%s, %s, %s, %s)'
try:
    session.execute(query, (1970, 'London', 'The Beatles', 'Memento Mori'))
    session.execute(query, (1970, 'The Moon', 'The Beatles', 'Laugh on me'))
except Exception as e:
    print(e)

In [7]:
# Create table
try:
    rows = session.execute("SELECT * FROM music_library WHERE YEAR = 1970")
except Exception as e:
    print(e)
for row in rows:
    print(', '.join([row.artist_name, str(row.year), row.album_name, row.city]))

The Beatles, 1970, Laugh on me, The Moon
The Beatles, 1970, Memento Mori, London


In [9]:
# Create table
try:
    rows = session.execute("SELECT * FROM music_library WHERE YEAR = 1970 AND ARTIST_NAME = 'The Beatles'")
except Exception as e:
    print(e)
for row in rows:
    print(', '.join([row.artist_name, str(row.year), row.album_name, row.city]))

The Beatles, 1970, Laugh on me, The Moon
The Beatles, 1970, Memento Mori, London


In [10]:
# Create table
try:
    rows = session.execute("SELECT * FROM music_library WHERE YEAR = 1970 AND ARTIST_NAME = 'The Beatles' AND ")
except Exception as e:
    print(e)
for row in rows:
    print(', '.join([row.artist_name, str(row.year), row.album_name, row.city]))

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


In [23]:
query = "DROP TABLE MUSIC_LIBRARY;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [24]:
session.shutdown()
cluster.shutdown()